In [2]:
import sys
sys.path.append('../')
import os
os.chdir('../')
%reload_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
%matplotlib notebook
%matplotlib inline
os.environ["CUDA_VISIBLE_DEVICES"]="1" #,
os.environ["CUDA_LAUNCH_BLOCKING"]="1"

In [3]:
import os 
import torch
import numpy as np

from torch.utils.tensorboard import SummaryWriter
from os.path import join as pjoin
from torch.distributions import Categorical
import json
import clip

import options.option_transformer as option_trans
import models.vqvae as vqvae
import utils.utils_model as utils_model
import utils.eval_trans as eval_trans
from dataset import dataset_TM_train
from dataset import dataset_TM_eval
from dataset import dataset_tokenize
import models.t2m_trans_uplow as trans
from options.get_eval_option import get_opt
from models.evaluator_wrapper import EvaluatorModelWrapper
import warnings
warnings.filterwarnings('ignore')
from exit.utils import get_model, visualize_2motions, generate_src_mask, uniform, cosine_schedule
from einops import rearrange, repeat
import os
import torch.nn.functional as F
# os.environ["CUDA_LAUNCH_BLOCKING"]="1"
# os.environ["CUDA_VISIBLE_DEVICES"]="4,5,6,7"

In [4]:
class Temp:
    def __init__(self):
        print('mock:: opt')
args = Temp()
args.dataname = args.dataset_name = 't2m'
args.down_t = 2

# train_loader_token = dataset_tokenize.DATALoader(args.dataname, 1, unit_length=2**args.down_t)

from utils.word_vectorizer import WordVectorizer
w_vectorizer = WordVectorizer('./glove', 'our_vab')
val_loader = dataset_TM_eval.DATALoader(args.dataname, False, 32, w_vectorizer)

dataset_opt_path = 'checkpoints/kit/Comp_v6_KLD005/opt.txt' if args.dataname == 'kit' else 'checkpoints/t2m/Comp_v6_KLD005/opt.txt'

wrapper_opt = get_opt(dataset_opt_path, torch.device('cuda'))
eval_wrapper = EvaluatorModelWrapper(wrapper_opt)

mock:: opt


 27%|██▋       | 401/1460 [00:00<00:00, 1902.52it/s]

100%|██████████| 1460/1460 [00:00<00:00, 1766.82it/s]


Pointer Pointing at 0
Reading checkpoints/t2m/Comp_v6_KLD005/opt.txt
Loading Evaluation Model Wrapper (Epoch 28) Completed!!


In [5]:
##### ---- Network ---- #####
clip_model, clip_preprocess = clip.load("ViT-B/32", device=torch.device('cuda'), jit=False)  # Must set jit=False for training
clip.model.convert_weights(clip_model)  # Actually this line is unnecessary since clip by default already on float16
clip_model.eval()
for p in clip_model.parameters():
    p.requires_grad = False

# https://github.com/openai/CLIP/issues/111
class TextCLIP(torch.nn.Module):
    def __init__(self, model) :
        super(TextCLIP, self).__init__()
        self.model = model
        
    def forward(self,text):
        return self.model.encode_text(text)
clip_model = TextCLIP(clip_model)

In [6]:
args.nb_code = 512 # 8192 # 
args.code_dim = 512 # 32 # 
args.output_emb_width = 512
args.down_t = 2
args.stride_t = 2
args.width = 512
args.depth = 3
args.dilation_growth_rate = 3
args.vq_act = 'relu'
args.vq_norm = None
args.quantizer = 'ema_reset'
args.mu = 0.99
args.clip_dim = 512
args.block_size = 51
args.n_head_gpt = 16
args.drop_out_rate = 0.1
args.ff_rate = 4


######## LARGE ########
args.num_layers = 9
args.embed_dim_gpt = 1024
args.resume_trans = '/home/epinyoan/git/MaskText2Motion/T2M-BD/output/t2m/2023-07-10-03-09-13_HML3D_33_upperLower_sepMask_Bilando_20Lyr/net_last.pth'
args.resume_pth = f'/home/epinyoan/git/MaskText2Motion/T2M-BD/output/vq/2023-07-02-03-11-31_16_VQVAE_upperlower_notShareCB/net_last.pth'

if args.dataset_name == 'kit':
    args.vq_name = 'VQVAE'
elif args.dataset_name == 't2m':
    args.vq_name = '2_VQVAE_HML3D'


args.exp_name = 'TEMP'
args.out_dir = 'output_GPT_Final'
args.out_dir = os.path.join(args.out_dir, f'{args.exp_name}')
args.vq_dir= f'output/{args.dataset_name}/{args.vq_name}'
args.vq_dir= '/home/epinyoan/git/MaskText2Motion/T2M-BD/output/vq/2023-07-02-03-11-31_16_VQVAE_upperlower_notShareCB'
codebook_dir = f'{args.vq_dir}/codebook/'
os.makedirs(args.out_dir, exist_ok = True)
os.makedirs(args.vq_dir, exist_ok = True)
os.makedirs(codebook_dir, exist_ok = True)
from models.vqvae_sep import VQVAE_SEP
net = VQVAE_SEP(args, ## use args to define different parameters in different quantizers
                       args.nb_code,
                       args.code_dim,
                       args.output_emb_width,
                       args.down_t,
                       args.stride_t,
                       args.width,
                       args.depth,
                       args.dilation_growth_rate)
trans_encoder = trans.Text2Motion_Transformer(num_vq=args.nb_code, 
                                    embed_dim=args.embed_dim_gpt, 
                                    clip_dim=args.clip_dim, 
                                    block_size=args.block_size, 
                                    num_layers=args.num_layers, 
                                    n_head=args.n_head_gpt, 
                                    drop_out_rate=args.drop_out_rate, 
                                    fc_rate=args.ff_rate)

ckpt = torch.load(args.resume_pth, map_location='cpu')
net.load_state_dict(ckpt['net'], strict=True)
# net = torch.nn.DataParallel(net)

ckpt = torch.load(args.resume_trans, map_location='cpu')
trans_encoder.load_state_dict(ckpt['trans'], strict=True)
trans_encoder = torch.nn.DataParallel(trans_encoder)
net.eval()
net.cuda()
trans_encoder.train()
trans_encoder.cuda()
''

''

### Train

In [6]:
args.batch_size = 2
num_workers = 8
train_loader = dataset_TM_train.DATALoader(args.dataname, args.batch_size, args.nb_code, codebook_dir, unit_length=2**args.down_t, num_workers=num_workers, up_low_sep=True)
train_loader_iter = dataset_TM_train.cycle(train_loader)

  3%|▎         | 601/23384 [00:00<00:07, 3081.47it/s]

100%|██████████| 23384/23384 [00:06<00:00, 3807.45it/s]


In [7]:
batch = next(iter(train_loader))
clip_text, target, m_tokens_len = batch

In [8]:
args.pkeep = .5
batch = next(train_loader_iter)
clip_text, target, m_tokens_len = batch
batch_size, max_len = target.shape[:2]
target, m_tokens_len = target.cuda(), m_tokens_len.cuda()
bs = target.shape[0]

text = clip.tokenize(clip_text, truncate=True).cuda()

feat_clip_text = clip_model(text).float()

# [INFO] Swap input index
if args.pkeep == -1:
    proba = np.random.rand(1)[0]
    mask = torch.bernoulli(proba * torch.ones(target.shape,
                                                        device=target.device))
else:
    mask = torch.bernoulli(args.pkeep * torch.ones(target.shape,
                                                        device=target.device))
# random only motion token (not pad token). To prevent pad token got mixed up.
seq_mask_no_end = generate_src_mask(max_len, m_tokens_len).unsqueeze(-1).repeat(1,1,2)
mask = torch.logical_or(mask, ~seq_mask_no_end).int()
r_indices = torch.randint_like(target, args.nb_code)
input_indices = mask*target+(1-mask)*r_indices

In [9]:
# Time step masking
mask_id = get_model(net).quantizer.nb_code + 2
rand_time = uniform((batch_size,), device = target.device)
rand_mask_probs = cosine_schedule(rand_time)
num_token_masked = (m_tokens_len * rand_mask_probs).round().clamp(min = 1)

# batch_randperm = torch.rand((batch_size, max_len), device = target.device).argsort(dim = -1)
seq_mask = generate_src_mask(max_len, m_tokens_len+1)
batch_randperm = torch.rand((batch_size, max_len, 2), device = target.device) - seq_mask_no_end.int()

# sort by parts
batch_randperm = batch_randperm.argsort(dim = 1)
mask_token = batch_randperm < rearrange(num_token_masked, 'b -> b 1 1')
print(mask_token.sum(1))
# sort all parts
# temp_shape = batch_randperm.shape
# batch_randperm = batch_randperm.view(batch_randperm.shape[0], -1).argsort(dim = 1)
# batch_randperm = batch_randperm.view(temp_shape)
# mask_token = batch_randperm < rearrange(num_token_masked*2, 'b -> b 1 1')
# print(mask_token.sum(1), num_token_masked)


masked_target = torch.where(mask_token, input=target, other=-1)
masked_input_indices = torch.where(mask_token, mask_id, input_indices)

att_txt = None #(torch.rand((seq_mask.shape[0], 1)) > 0.1)
cls_pred = trans_encoder(masked_input_indices, feat_clip_text, seq_mask, att_txt, m_tokens_len)[:, 1:]
cls_pred = cls_pred.contiguous()

# # check that the random applies only actual seq not the blank
torch.equal((mask_token * seq_mask.unsqueeze(-1)), mask_token)

tensor([[34, 34],
        [30, 30]], device='cuda:0')


True

In [10]:
### Loss
weights = seq_mask_no_end / (seq_mask_no_end.sum(1).unsqueeze(1) * seq_mask_no_end.shape[0])/2
cls_pred_seq_masked = torch.masked_select(cls_pred, seq_mask_no_end.unsqueeze(-1)).view(-1, cls_pred.shape[-1])
target_seq_masked = torch.masked_select(target, seq_mask_no_end)
weight_seq_masked = torch.masked_select(weights, seq_mask_no_end)
loss_cls = F.cross_entropy(cls_pred_seq_masked, target_seq_masked, reduction = 'none')
loss_cls = (loss_cls * weight_seq_masked).sum()
loss_cls

tensor(0.4090, device='cuda:0', grad_fn=<SumBackward0>)

#### Upper Lower Together

In [11]:
batch_randperm = torch.rand((batch_size, max_len, 1), device = target.device) - seq_mask_no_end[..., :1].int()
batch_randperm = batch_randperm.argsort(dim = 1)
mask_token = batch_randperm < rearrange(num_token_masked, 'b -> b 1 1')
mask_token = mask_token.repeat(1, 1, 2)
mask_token.shape

torch.Size([2, 50, 2])

### Eval

In [13]:
val_loader = dataset_TM_eval.DATALoader(args.dataname, True, 32, w_vectorizer)

 28%|██▊       | 406/1460 [00:00<00:00, 1998.20it/s]

100%|██████████| 1460/1460 [00:01<00:00, 1389.17it/s]

Pointer Pointing at 0


In [8]:
class LoggerWriterMock:
    def __init__(self):
        self.info
    def info(self, *args):
        print(*args)
    def add_scalar(self, *args):
        print(*args)
    def add_video(self, *args):
        print(*args)
logger = LoggerWriterMock()
logger.info('test')
writer = LoggerWriterMock()
writer.add_scalar('./Test/FID', 'fid', 'nb_iter')

test
./Test/FID fid nb_iter


In [9]:
########## Test Single Sample ##########
# word_embeddings, pos_one_hots, clip_text, sent_len, pose, m_length, token, name = next(iter(val_loader))
# seq = pose.shape[1]
# m_length = m_length.cuda()
m_length = torch.tensor([180, 54]).cuda() # length: [196, 54]
m_tokens_len = torch.ceil((m_length)/4)
feat_clip_text = torch.rand([2, 512]) # [2, 512]
feat_clip_text = feat_clip_text.cuda()
import timeit
total = 0
num_all = 0
blank_id = get_model(trans_encoder).num_vq
for k in range(1):
    start = timeit.default_timer()
    index_motion, temp = trans_encoder(feat_clip_text, type="sample", m_length=m_length, if_test=True, CFG=-1)
    pred_pose = net(index_motion[k:k+1, :int(m_tokens_len[k].item())], type='decode')

    stop = timeit.default_timer()
    total += (stop - start)
    num_all += 1
total/num_all


0.6192212207242846

In [13]:
pred_pose_eval, pose, m_length, clip_text, best_fid, best_iter, best_div, best_top1, best_top2, best_top3, best_matching, best_multi, writer, logger = \
    eval_trans.evaluation_transformer(args.out_dir, 
        val_loader, 
        net, 
        trans_encoder, 
        logger, 
        writer, 
        0, 
        best_fid=1000, 
        best_iter=0, 
        best_div=100, 
        best_top1=0, 
        best_top2=0, 
        best_top3=0, 
        best_matching=100, 
        clip_model=clip_model, 
        eval_wrapper=eval_wrapper,
        num_repeat=11,
        rand_pos=True,
        CFG=-1)

  0%|          | 0/145 [00:00<?, ?it/s]

100%|██████████| 145/145 [42:56<00:00, 17.77s/it]


--> 	 Eva. Iter 0 :, 
                FID. 0.3078 , 
                Diversity Real. 9.3197, 
                Diversity. 9.2266, 
                R_precision_real. [0.51400862 0.70280172 0.80344828], 
                R_precision. [0.49590517 0.68987069 0.78060345], 
                matching_score_real. 2.9507390877296187, 
                matching_score_pred. 3.0880776471105116, 
                multimodality. 0.9272
./Test/FID 0.3078417072668742 0
./Test/Diversity 9.226602 0
./Test/top1 0.4959051724137931 0
./Test/top2 0.6898706896551724 0
./Test/top3 0.780603448275862 0
./Test/matching_score 3.0880776471105116 0
./Test/multimodality 0.9271635 0
--> --> 	 FID Improved from 1000.00000 to 0.30784 !!!
--> --> 	 matching_score Improved from 100.00000 to 3.08808 !!!
--> --> 	 Diversity Improved from 100.00000 to 9.22660 !!!
--> --> 	 Top1 Improved from 0.0000 to 0.4959 !!!
--> --> 	 Top2 Improved from 0.0000 to 0.6899 !!!
--> --> 	 Top3 Improved from 0.0000 to 0.7806 !!!


### Visualization

In [10]:
from exit.utils import get_model, visualize_2motions

In [15]:
for word_embeddings, pos_one_hots, clip_text, sent_len, pose, m_length, token, name in val_loader:
    b = False
    for k, text in enumerate(clip_text):
        if 'a man steps forward and does a handstand.' in text:
        # if 'a man rises from the ground, walks in a circle and sits back down on the ground' in text:
            print('ok', k)
            print(clip_text[k])
            b = True
            break
    if b:
        print('---')
        break

In [87]:
rand_pos = True
CFG = -1
m_tokens_len = torch.ceil((m_length)/4)
text = clip.tokenize(clip_text, truncate=True).cuda()
feat_clip_text = clip_model(text).float()
index_motion = trans_encoder(feat_clip_text, type="sample", m_length=m_length.cuda(), rand_pos=rand_pos, CFG=CFG)
pred_pose = net(index_motion[k:k+1, :int(m_tokens_len[k].item())], type='decode')

In [88]:
m_tokens_len[k], pose[k].shape

(tensor(35.), torch.Size([196, 263]))

In [90]:
x = pose[k][:m_length[k]].detach().cpu().numpy()
y = pred_pose[0].detach().cpu().numpy()
l = m_length[k]
caption = clip_text[k]
visualize_2motions(x, val_loader.dataset.std, val_loader.dataset.mean, args.dataname, l, y, save_path=None) # 'study/a man steps forward and does a handstand.html'